In [36]:
import sqlite3
import pandas as pd
import re

# Path to your SQLite database (use ":memory:" for an in-memory database)
db_path = "../fredy/db/listings.db"

try:
    # Connect to SQLite database
    connection = sqlite3.connect(db_path)
    print("Successfully connected to SQLite")

    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()

    # Query to list all tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    # Fetch all results
    tables = cursor.fetchall()

    """
    # Print the table names
    print("Tables in the database:")
    for table in tables:
        print(table[0])  # Each result is a tuple; table name is the first element
    """

    # Specify the table name
    table_name = "listing"

    # Query to get the table's structure
    cursor.execute(f"PRAGMA table_info({table_name});")

    # Fetch all results
    columns = cursor.fetchall()

    # Print column information
    print(f"Attributes of table '{table_name}':")
    print("CID | Name | Type | NotNull | Default | PrimaryKey")
    for column in columns:
        print(column)  # Each column is a tuple
    
    # Example: Query data
    query = (f"SELECT * FROM {table_name}")

    df = pd.read_sql_query(query, connection).query('jobKey != "TestJob"').drop(columns = ['jobKey', 'id']).drop_duplicates()
    df['PLZ'] = df['address'].transform(lambda x: re.search("[0-9]{5}", x)[0])
    print(df['PLZ'])
        
except sqlite3.Error as e:
    print(f"SQLite error: {e}")
finally:
    # Close the connection
    if connection:
        connection.close()
        print("SQLite connection closed")

Successfully connected to SQLite
Attributes of table 'listing':
CID | Name | Type | NotNull | Default | PrimaryKey
(0, 'serviceName', 'TEXT', 0, None, 0)
(1, 'jobKey', 'TEXT', 0, None, 0)
(2, 'id', 'TEXT', 0, None, 0)
(3, 'size', 'TEXT', 0, None, 0)
(4, 'rooms', 'TEXT', 0, None, 0)
(5, 'price', 'TEXT', 0, None, 0)
(6, 'address', 'TEXT', 0, None, 0)
(7, 'title', 'TEXT', 0, None, 0)
(8, 'link', 'TEXT', 0, None, 0)
(9, 'description', 'TEXT', 0, None, 0)
1      10719
2      10969
3      13585
4      13409
5      13507
       ...  
132    13585
133    13595
134    13585
135    10781
136    13409
Name: PLZ, Length: 111, dtype: object
SQLite connection closed


In [44]:
import requests

# URL for fetching JSON data
url = 'https://v6.bvg.transport.rest/journeys?from.latitude=52.543333&from.longitude=13.351686&from.address=12623+Berlin&to=900014101&departure=tomorrow+2pm&results=2'

try:
    # Make a GET request to the URL
    response = requests.get(url)

    # Raise an exception if the request was not successful
    response.raise_for_status()

    # Parse the JSON data from the response
    data = response.json()

except requests.exceptions.RequestException as e:
    # Handle any errors that occur during the request
    print(f"An error occurred: {e}")

In [45]:
from datetime import datetime
import jmespath
from pprint import pprint

# data = {"key1": {"key2": {"key3": "value"}}}
result = datetime.fromisoformat(jmespath.search('journeys[0].legs[0].arrival', data))-datetime.fromisoformat(jmespath.search('journeys[0].legs[0].departure', data))
pprint(result)

datetime.timedelta(seconds=180)


In [46]:
pprint(jmespath.search('journeys[0].legs[0]', data))

{'arrival': '2024-12-24T14:05:00+01:00',
 'arrivalDelay': None,
 'departure': '2024-12-24T14:02:00+01:00',
 'departureDelay': None,
 'destination': {'id': '900009101',
                 'location': {'id': '900009101',
                              'latitude': 52.542201,
                              'longitude': 13.34953,
                              'type': 'location'},
                 'name': 'U Amrumer Str. (Berlin)',
                 'products': {'bus': True,
                              'express': False,
                              'ferry': False,
                              'regional': False,
                              'suburban': False,
                              'subway': True,
                              'tram': False},
                 'type': 'stop'},
 'distance': 259,
 'origin': {'address': '12623 Berlin',
            'id': None,
            'latitude': 52.543333,
            'longitude': 13.351686,
            'type': 'location'},
 'plannedArrival': '2024-12-

In [40]:
# ISO 8601 datetime string
datetime_str = result

# Parse the datetime string
parsed_datetime = datetime.fromisoformat(datetime_str)

NameError: name 'result' is not defined

In [25]:
parsed_datetime

datetime.datetime(2024, 12, 23, 14, 4, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))

In [76]:
from geopy.geocoders import Nominatim
Address = "Berlin, Germany"
geolocator = Nominatim(user_agent="name_of_the_agent")
location = geolocator.geocode(Address, namedetails=True)
print((location.latitude, location.longitude))

(52.510885, 13.3989367)


In [68]:
flat_locations = {
    'Berliner Rechnungshof': {'latitude': 52.524121062023426, 'longitude': 13.352456193718531}
}

poi = {
    'HTW Berlin': {'latitude': 52.4563126, 'longitude': 13.5294627},
    'BHT Berlin': {'latitude': 52.5442824, 'longitude': 13.353129769195922}
}

In [77]:
import folium
from IPython.display import display

# Example dictionary of latitudes and longitudes
locations = poi

# Create a map centered around the first location
# first_location = next(iter(locations.values()))
mymap = folium.Map(location=[52.510885, 13.3989367], zoom_start=11)

# Add markers to the map
for name, coords in locations.items():
    folium.Marker(
        location=[coords["latitude"], coords["longitude"]],
        popup=name
    ).add_to(mymap)

for name, coords in flat_locations.items():
    folium.Marker(
        location=[coords["latitude"], coords["longitude"]],
        popup=name,
        icon=folium.Icon(color='green')
    ).add_to(mymap)

# Display the map in the Jupyter Notebook
display(mymap)

In [128]:
def travel_time(origin, destination):
    
    url = f'https://v6.bvg.transport.rest/journeys?from.latitude={origin['latitude']}&from.longitude={origin['longitude']}&from.address={'Start'}&to.latitude={destination['latitude']}&to.longitude={destination['longitude']}&to.address={'Ende'}&departure=tomorrow+2pm&results=1'

    try:
        # Make a GET request to the URL
        response = requests.get(url)
    
        # Raise an exception if the request was not successful
        response.raise_for_status()
    
        # Parse the JSON data from the response
        data = response.json()
        # pprint(jmespath.search('journeys[0].legs', data))

        # is there always a legs option?
        travel_time = datetime.fromisoformat(jmespath.search('journeys[0].legs[-1].arrival', data))-datetime.fromisoformat(jmespath.search('journeys[0].legs[0].departure', data))
        return(travel_time)
    
    except requests.exceptions.RequestException as e:
        # Handle any errors that occur during the request
        print(f"An error occurred: {e}")

In [129]:
for item in poi.items():
    seconds = travel_time(item[1], flat_locations['Berliner Rechnungshof']).total_seconds()
    minutes = seconds // 60
    seconds = (seconds % 60)
    print(f'{item[0]}: {int(minutes)} min {int(seconds)} s')

HTW Berlin: 58 min 0 s
BHT Berlin: 19 min 0 s
